In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

# Model Function

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
pd.options.mode.chained_assignment = None
def prepare_data_main_model(df, seq_length, output_size, date_range):
    '''
    df: pandas df contain all the data
    seq_length: number of days consider as input
    output_size: number of days to predict
    date_range: length of history to consider
    Output:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - state_ordered: List of unique states in the data.
    '''
    full_data = []
    state_ordered = []

    for state in df.index.get_level_values('fips').unique():
        df_state = df.iloc[df.index.get_level_values('fips') == state]
 # If the length of data is less than or equal to date_range, consider all data
        if len(df_state) <= date_range:
            L = len(df_state.to_numpy())
            train_state = []
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
# If data is larger than date_range, consider the most recent date_range data
        else:
            df_state = df.iloc[df.index.get_level_values('fips') == state][-date_range:]

            train_state = []

            L = len(df_state.to_numpy())
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
    return full_data, state_ordered

def splitdata(full_data, ratio, batch_size):
    '''
    Split the prepared data into training and testing datasets.

    Parameters:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - ratio: Ratio of data to be used for training (e.g., 0.8 for 80% training, 20% testing).
    - batch_size: Size of mini-batches for DataLoader.

    Output:
    - train_loader: DataLoader for the training dataset.
    - test_loader: DataLoader for the testing dataset.
    '''
    train_size = int(ratio * len(full_data))
    test_size = len(full_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_data, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True, drop_last=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              shuffle=True, drop_last=True)

    return train_loader, test_loader


class LSTM(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size

        self.hidden_layer_size = hidden_layer_size

        self.num_layers = num_layers

        self.output_size = output_size

        self.lstm = nn.LSTM(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)

        self.dropout = nn.Dropout(dropout_rate)

        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = (torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device),
             torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device))

        lstm_out, self.hidden_cell = self.lstm(input_seq, h)

        # only return the results for last sequence
        lstm_out = lstm_out[:, -1, :]
        predictions = self.linear(lstm_out)
        #predictions = F.relu(predictions)
        #predictions = F.softplus(predictions)
        predictions = F.leaky_relu(predictions, negative_slope=0.01)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions

 ### Training

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F

def LSTM_train(test_week,option):

    data_file = '/content/drive/MyDrive/Flu Forecasting/code/smoothed_rate_withcovidww.csv'

    model_file = '/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US model/'
    prediction_file = '/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US result/'
    summary_file ='/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US result/'
    def perdelta(start, end, delta):
        curr = start
        while curr < end:
            yield curr
            curr += delta
    global device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # perform grid search to find a best model
    if option == 0:
      params = OrderedDict(
          target = ['flu_rate'], ### Our target variable
          lr = [0.0001,0.0005],
          batch_size = [2],
          seq_length = [4,6,8,10], # length of input sequence to predict values
          output_size = [4], # we are predicting next four weeks admissions
          num_pred_features = [1],
          week_range = [300], # number of historical data (per state) used to train the model
          input_size = [15], # total number of input features
          hidden_layer_size = [32,64,128], #test
          num_layers = [1], #test
          ratio = [0.8],  # ratio of training set and validation set
          num_epochs = [50],
          dropout_rate = [0.8],
          lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                      ]
      )


      class RunBuilder():
          @staticmethod
          def get_runs(params):

              Run = namedtuple('Run', params.keys())

              runs = []
              for v in product(*params.values()):
                  runs.append(Run(*v))

              return runs

      pd.options.mode.chained_assignment = None

      runs = RunBuilder.get_runs(params)
#directly run the best model
    elif option == 1:
      params = OrderedDict(
          target = ['rate'], ### Our target variable
          lr = [0.0001],
          batch_size = [2],
          seq_length = [8], # length of input sequence to predict values
          output_size = [4], # we are predicting next four weeks admissions
          num_pred_features = [1],
          week_range = [64], # number of historical data (per state) used to train the model
          input_size = [3], # total number of input features
          hidden_layer_size = [128], #test
          num_layers = [1], #test
          ratio = [0.8],  # ratio of training set and validation set
          num_epochs = [50],
          dropout_rate = [0.8],
          lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                      ]
      )


      class RunBuilder():
          @staticmethod
          def get_runs(params):

              Run = namedtuple('Run', params.keys())

              runs = []
              for v in product(*params.values()):
                  runs.append(Run(*v))

              return runs

      pd.options.mode.chained_assignment = None

      runs = RunBuilder.get_runs(params)




    # output dict
    columns = [
        'Model',
        'lr',
        'batch_size',
        'seq_length',
        #'week_range',
        'hidden_layer_size',
        'num_layers',
        'lossfunc',
        'mae_validation',
        'mae_validation_1w',
        'mae_validation_2w',
        'mae_validation_3w',
        'mae_validation_4w',
        'Target'
    ]
    # output dataframe
    df_summary = pd.DataFrame(columns=columns)


    pd.options.mode.chained_assignment = None



    for run in runs:
        if run.target == 'flu_rate':
            results_file = 'flu_rate'

        validation_predictions = []
        validation_labels = []
        # read data
        df = pd.read_csv(data_file)
        #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
        df['Week_end'] = pd.to_datetime(df['Week_end'])
        # too many missing values in State'11'
        df =  df[df['fips'] != 11]
        df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA','s05_value','scontrol_value'], axis=1)
    # select region
        df['fips'] = df['fips'].astype(str).str.zfill(2)
        #df = df[['fips','Week_end','difference','rate','smoothed_admission']]
        # test set is the week from 2024-01-20
        start_date1 = pd.to_datetime(test_week) - timedelta(weeks=run.seq_length )
        end_date1 = pd.to_datetime(test_week) + timedelta(weeks= 1 )


        df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < end_date1)] # Use data after 2023-11-01 as testing set
        df_test = df_test.set_index(['fips', 'Week_end'])
        df_test1 = df_test.copy()
        #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
        df_train = df[(df['Week_end'] < pd.to_datetime(test_week) )& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
        #df_train = df[(df['Week_end'] < pd.to_datetime('2023-05-01') )& (df['Week_end'] >pd.to_datetime('2022-08-01')) ]
        df_train = df_train.set_index(['fips', 'Week_end'])

        #rescale the data from 0 to 1, normalization
        first_col = df_train.pop(run.target)
        df_train.insert(0, run.target, first_col)
        first_col_2 = df_test.pop(run.target)
        df_test.insert(0, run.target, first_col_2)

        scaler = MinMaxScaler(feature_range=(0, 1))

        scaler.fit(df_train.iloc[:, 1:])
        train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
        test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

        scaler_target = MinMaxScaler(feature_range=(0, 1))
        scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
        train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
        test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))
        #train_target_normalized = np.log(df_train.iloc[:, 0] + 1)
        #test_target_normalized = np.log(df_test.iloc[:, 0] + 1)

        df_train.iloc[:, 1:] = train_features_normalized
        df_train.iloc[:, 0] = train_target_normalized
        df_test.iloc[:, 1:] = test_feature_normalized
        df_test.iloc[:, 0] = test_target_normalized

        '''
        Training
        '''
        # prepare the training and validation data (sequance format)
        full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                                run.output_size, run.week_range)

        model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size ,
                          run.dropout_rate).to(device)
        # split the training data into training set and validation set
        train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

        loss_function = run.lossfunc
      # add l2 regularization
        optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr,weight_decay=1e-5)

        track_loss_train = []
        track_loss_test = []
        best_loss = 100000
        ###

        best_mae_numerical = None
        mae_numerical_weekly = [None] * 4

        best_mae_numerical_weekly = [None] * 4
        for i in trange(run.num_epochs):
    # Set the model in training mode
            model_main.train()
            epoch_loss_train = 0
    # Iterate over the training data
            for i, (seq, labels) in enumerate(train_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                optimizer_main.zero_grad()
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                # Initialize the hidden state of the LSTM
                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device),
                              torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device))

                # Forward pass to get predictions
                y_pred = model_main(seq.float())

                # Calculate the loss and perform backpropagation
                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                single_loss.backward()
                optimizer_main.step()

                epoch_loss_train += single_loss.item()
            # Track the training loss for this epoch
            track_loss_train.append(epoch_loss_train)

            with torch.no_grad():
              # Set the model in evaluation mode
                model_main.eval()
                epoch_loss_test = 0
                ########
                # Initialize lists to store validation predictions and labels
                validation_predictions = []
                validation_labels = []
                for i, (seq, labels) in enumerate(test_loader_main):
                    seq, labels = seq.to(device), labels.to(device)
                    seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                    model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                    torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                    y_pred = model_main(seq.float())

                    single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                    epoch_loss_test += single_loss.item()
                    ######
                    # Store predictions and labels for later evaluation
                    validation_predictions.extend(y_pred.cpu().numpy())
                    validation_labels.extend(labels.cpu().numpy())
                track_loss_test.append(epoch_loss_test)
            # Combine validation predictions and labels, and perform inverse scaling to get true value
            validation_predictions = np.concatenate(validation_predictions, axis=0)
            validation_labels = np.concatenate(validation_labels, axis=0)
            validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
            validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)
            #validation_predictions = (np.exp(validation_predictions) - 1).reshape(-1, 4)
            #validation_labels = (np.exp(validation_labels) - 1).reshape(-1, 4)
            # Calculate total and weekly MSE and accuracy for validation set
            #mse_numerical = np.mean((validation_predictions - validation_labels)**2)
            mae_numerical = np.mean(np.abs(validation_predictions - validation_labels))
            for week in range(4):
                # Calculate weekly MSE
                #mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
                #mse_numerical_weekly[week] = mse_numerical_week
                mae_numerical_week = np.mean(np.abs(validation_predictions[:, week] - validation_labels[:, week]))
                mae_numerical_weekly[week] = mae_numerical_week
            if epoch_loss_test  < best_loss:
              # Update the best loss and save the model; perform early stopping
                best_mae_numerical = mae_numerical
                best_mae_numerical_weekly = mae_numerical_weekly

                best_loss = epoch_loss_test
                print('Train Loss: ', epoch_loss_train)
                print('Test Loss: ', epoch_loss_test)
                es = 0
                torch.save(model_main.state_dict(),
                              model_file + run.target +'_' +test_week+'_'  +
                              str(run.lr) + '_' + str(run.hidden_layer_size)\
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                              + '_LSTM_weights.pt')
            else:
                es += 1
                print("Counter {} of 5".format(es))
                print('Train Loss: ', epoch_loss_train)
                print('Test Loss: ', epoch_loss_test)


            if es > 4:
                print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                          epoch_loss_test,
                          "...")

                break

        '''
        Prediction
        '''
        df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                          'Prediction_2w', 'Prediction_3w',
                                          'Prediction_4w'])
      #test set starts with 2023-11-01
        test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                          >= pd.to_datetime(test_week)].index.get_level_values('Week_end').unique()
        test_states = df_test.index.get_level_values('fips').unique()

        m_state_dict_main = torch.load(model_file + run.target+'_' +test_week+ '_' +
                                      str(run.lr) + '_' + str(run.hidden_layer_size) \
                                      + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                      + '_LSTM_weights.pt')

        model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                          run.output_size, run.dropout_rate).to(device)
        model_main.load_state_dict(m_state_dict_main)

        with torch.no_grad():
            #model_main.eval()
            for fips in test_states:
                for week in test_weeks:
                    seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                    & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                    seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                    model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                    torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                    prediction = model_main(seq.float())

                    prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                    #prediction = np.exp(prediction.cpu().detach().numpy().reshape(-1, 1)) - 1
                    # save prediction results
                    dic = {
                        'fips' : fips,
                        'Week_end' : week,
                        'Prediction_1w' : prediction[0].item(),
                        'Prediction_2w' : prediction[1].item(),
                        'Prediction_3w' : prediction[2].item(),
                        'Prediction_4w' : prediction[3].item()

                    }

                    df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


            df_output.to_csv(prediction_file + run.target + '_'  +
                                str(run.lr) + '_' + str(run.hidden_layer_size) \
                                + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                + '.csv')

            '''
            Evaluation
            '''





            dic_lstm = {
                'Model': 'LSTM',
                'lr': run.lr,
                'batch_size': run.batch_size,
                'seq_length': run.seq_length,
                #'week_range': run.week_range,
                'hidden_layer_size': run.hidden_layer_size,
                'num_layers': run.num_layers,
                'lossfunc': run.lossfunc,

                'mae_validation': best_mae_numerical,

                'mae_validation_1w': best_mae_numerical_weekly[0],
                'mae_validation_2w': best_mae_numerical_weekly[1],
                'mae_validation_3w': best_mae_numerical_weekly[2],
                'mae_validation_4w': best_mae_numerical_weekly[3],

                'Target': run.target
            }

            print('mae_validation:')
            print(dic_lstm['mae_validation'])


            df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

    df_summary.to_csv(summary_file  +test_week+ 'summarytest.csv')
    df_summary = df_summary.sort_values(by='mae_validation', ascending=True)
    return df_summary,df_output

###Forecasting

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


def LSTM_Prediction(test_week,variable_combinations):
    n_result = pd.DataFrame()
    data_file = '/content/drive/MyDrive/Flu Forecasting/code/smoothed_rate_withcovidww.csv'

    model_file = '/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US model/'
    prediction_file = '/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US result/1000 times/'
    summary_file ='/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US result/1000 times/'
    global device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def perdelta(start, end, delta):
        curr = start
        while curr < end:
            yield curr
            curr += delta
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    constant_params = {
        'target': 'flu_rate',
        'batch_size': 2,
        'input_size': 15, # total number of features
        'output_size': 4, # number of weeks to predict
        'week_range': 300,
        'num_pred_features': 1,
        'ratio': 0.8,
        'num_epochs':50,
        'num_layers': 1,
        'dropout_rate': 0.8,
        'lossfunc': nn.SmoothL1Loss(beta=0.05, reduction = 'sum')


    }


    class RunBuilder():
        @staticmethod
        def get_runs(constant_params, variable_combinations):
            # Ensure variable_combinations is a list containing a single dictionary
            if isinstance(variable_combinations, dict):
                variable_combinations = [variable_combinations]

            # Dynamically create a Run namedtuple based on keys
            Run = namedtuple('Run', list(constant_params.keys()) + list(variable_combinations[0].keys()))

            runs = []
            for combo in variable_combinations:
                # Merge the constant and variable parameters
                params = {**constant_params, **combo}
                runs.append(Run(**params))

            return runs

    # Generate runs with selected combinations
    runs = RunBuilder.get_runs(constant_params, variable_combinations)

    pd.options.mode.chained_assignment = None
    for run in runs:
        if run.target == 'flu_rate':
            results_file = 'flu_rate'

        validation_predictions = []
        validation_labels = []
        # read data
        df = pd.read_csv(data_file)
        #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
        df['Week_end'] = pd.to_datetime(df['Week_end'])
        # too many missing values in State'11'
        df =  df[df['fips'] != 11]
        df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA','s05_value','scontrol_value'], axis=1)
    # select region
        df['fips'] = df['fips'].astype(str).str.zfill(2)
        #df = df[['fips','Week_end','difference','rate','smoothed_admission']]

        # test set is the week from 2024-01-20
        start_date1 = pd.to_datetime(test_week) - timedelta(weeks=run.seq_length )
        end_date1 = pd.to_datetime(test_week) + timedelta(weeks= 1 )
        df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < end_date1)] # Use data after 2023-11-01 as testing set
        df_test = df_test.set_index(['fips', 'Week_end'])
        df_test1 = df_test.copy()
        #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
        df_train = df[(df['Week_end'] < pd.to_datetime(test_week) )& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set

        df_train = df_train.set_index(['fips', 'Week_end'])

        #rescale the data from 0 to 1, normalization
        first_col = df_train.pop(run.target)
        df_train.insert(0, run.target, first_col)
        first_col_2 = df_test.pop(run.target)
        df_test.insert(0, run.target, first_col_2)

        scaler = MinMaxScaler(feature_range=(0, 1))

        scaler.fit(df_train.iloc[:, 1:])
        train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
        test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

        scaler_target = MinMaxScaler(feature_range=(0, 1))
        scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
        train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
        test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))
        #train_target_normalized = np.log(df_train.iloc[:, 0] + 1)
        #test_target_normalized = np.log(df_test.iloc[:, 0] + 1)

        df_train.iloc[:, 1:] = train_features_normalized
        df_train.iloc[:, 0] = train_target_normalized
        df_test.iloc[:, 1:] = test_feature_normalized
        df_test.iloc[:, 0] = test_target_normalized
        '''
        desired_order = ['rate', 'difference', 'smoothed_admission'] + [col for col in df_train.columns if col not in ['rate', 'difference', 'smoothed_admission']]
        df_train = df_train[desired_order]
        df_test = df_test[desired_order]



        #rescale the data from 0 to 1, normalization
        scaler_general = MinMaxScaler(feature_range=(0, 1))
        scaler_diff_smooth = MinMaxScaler(feature_range=(0, 100))
        scaler_target = MinMaxScaler(feature_range=(0, 1))

        # Assuming the first column 'target' is your target variable
        # Scale the target variable
        train_target_normalized = scaler_target.fit_transform(df_train.iloc[:, 0].values.reshape(-1, 1))
        test_target_normalized = scaler_target.transform(df_test.iloc[:, 0].values.reshape(-1, 1))

        # Scale 'difference' and 'smoothed_admission' with a specific range [0, 10]
        scaler_diff_smooth.fit(df_train.iloc[:, 1:3])  # Now correctly indexes 'difference' and 'smoothed_admission'
        train_diff_smooth_normalized = scaler_diff_smooth.transform(df_train.iloc[:, 1:3])
        test_diff_smooth_normalized = scaler_diff_smooth.transform(df_test.iloc[:, 1:3])

        # Scale the rest of the features (excluding target and the two specific features) normally
        scaler_general.fit(df_train.iloc[:, 3:])  # Correctly indexes the remaining features
        train_features_normalized = scaler_general.transform(df_train.iloc[:, 3:])
        test_features_normalized = scaler_general.transform(df_test.iloc[:, 3:])

        # Reassemble the dataframes
        df_train.iloc[:, 0] = train_target_normalized.ravel()  # Target column
        df_train.iloc[:, 1:3] = train_diff_smooth_normalized  # 'difference' and 'smoothed_admission'
        df_train.iloc[:, 3:] = train_features_normalized  # The rest of the features

        df_test.iloc[:, 0] = test_target_normalized.ravel()
        df_test.iloc[:, 1:3] = test_diff_smooth_normalized
        df_test.iloc[:, 3:] = test_features_normalized
        '''
        for i in range(1000):
          df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                            'Prediction_2w', 'Prediction_3w',
                                            'Prediction_4w'])
        #test set starts with 2024-01-06
          test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                              >= pd.to_datetime(test_week)].index.get_level_values('Week_end').unique()
          test_states = df_test.index.get_level_values('fips').unique()

          m_state_dict_main = torch.load(model_file + run.target +'_' +test_week+ '_'  +
                                          str(run.lr) + '_' + str(run.hidden_layer_size) \
                                          + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                          + '_LSTM_weights.pt')

          model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                            run.output_size, run.dropout_rate).to(device)
          model_main.load_state_dict(m_state_dict_main)

          with torch.no_grad():
              #model_main.eval()
              for fips in test_states:
                  for week in test_weeks:
                      seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                        & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                      seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                      model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                        torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                      prediction = model_main(seq.float())

                      prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                      #prediction = np.exp(prediction.cpu().detach().numpy().reshape(-1, 1)) - 1
                      # save prediction results
                      dic = {
                          'fips' : fips,
                          'Week_end' : week,
                          'Prediction_1w' : prediction[0].item(),
                          'Prediction_2w' : prediction[1].item(),
                          'Prediction_3w' : prediction[2].item(),
                          'Prediction_4w' : prediction[3].item()

                      }

                      df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


              df_output.to_csv(prediction_file + run.target +'_' +test_week+ '_'  +
                                  str(run.lr) + '_' + str(run.hidden_layer_size) \
                                  + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                  + '.csv')
              n_result= pd.concat([n_result, df_output], ignore_index=True)



    n_result = n_result.sort_values(by='fips')
    n_result.to_pickle(summary_file + test_week+ '_allresult.pkl')
    return n_result

#Forecasting

##model selection

In [ ]:
from datetime import datetime, timedelta


start_date = datetime.strptime('2023-10-07', '%Y-%m-%d')
end_date = datetime.strptime('2024-02-17', '%Y-%m-%d')
current_date = start_date
dates_list = []

while current_date <= end_date:
    dates_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=7)
dates_list

['2024-01-13',
 '2024-01-20',
 '2024-01-27',
 '2024-02-03',
 '2024-02-10',
 '2024-02-17']

In [ ]:
from google.colab import output
test_weeks = dates_list
option = 0
for week in test_weeks:
    summary_table, b = LSTM_train(week, option)
    output.clear()

In [ ]:

#test_week = '2023-10-01'
#test_week = '2023-11-01'
#test_week = '2023-12-01'
test_week = '2023-09-30'
#test_week = '2024-02-01'
option = 0
summarytable,b = LSTM_train(test_week,option)
summarytable

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  2%|▏         | 1/50 [00:02<02:06,  2.59s/it]

Train Loss:  533.8468897119164
Test Loss:  64.75391590129584


  4%|▍         | 2/50 [00:05<02:08,  2.67s/it]

Train Loss:  341.4048210764304
Test Loss:  58.83299034598167


  6%|▌         | 3/50 [00:08<02:06,  2.70s/it]

...

Counter 4 of 5
Train Loss:  113.64201565776602
Test Loss:  33.38978977734223


 24%|██▍       | 12/50 [00:35<01:52,  2.97s/it]

Counter 5 of 5
Train Loss:  116.98026511908392
Test Loss:  28.598745115334168
Early stopping with best_loss:  27.52471433527535 and test_loss for this epoch:  28.598745115334168 ...
mae_validation:
0.5333742985039519



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


,Model,lr,batch_size,seq_length,hidden_layer_size,num_layers,lossfunc,mae_validation,mae_validation_1w,mae_validation_2w,mae_validation_3w,mae_validation_4w,Target
3,LSTM,0.0001,2,4,256,1,SmoothL1Loss(),0.426852,0.352895,0.395088,0.530987,0.591012,rate
13,LSTM,0.0001,2,10,64,1,SmoothL1Loss(),0.435373,0.329440,0.404648,0.453385,0.543349,rate
26,LSTM,0.0005,2,8,128,1,SmoothL1Loss(),0.435505,0.369385,0.450823,0.557627,0.656678,rate
14,LSTM,0.0001,2,10,128,1,SmoothL1Loss(),0.436795,0.378866,0.480702,0.577748,0.567730,rate
2,LSTM,0.0001,2,4,128,1,SmoothL1Loss(),0.439042,0.346694,0.401489,0.469782,0.580520,rate
1,LSTM,0.0001,2,4,64,1,SmoothL1Loss(),0.440515,0.303749,0.409141,0.491859,0.594494,rate
11,LSTM,0.0001,2,8,256,1,SmoothL1Loss(),0.457840,0.311857,0.439973,0.503870,0.588642,rate
27,LSTM,0.0005,2,8,256,1,SmoothL1Loss(),0.460310,0.352440,0.405706,0.487385,0.639533,rate
19,LSTM,0.0005,2,4,256,1,SmoothL1Loss(),0.462349,0.378433,0.419067,0.513607,0.603954,rate
20,LSTM,0.0005,2,6,32,1,SmoothL1Loss(),0.463938,0.397804,0.412176,0.506332,0.606075,rate


In [ ]:
df_summary = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US result/2023-12-01summarytest.csv')
df_summary = df_summary.sort_values(by='mae_validation', ascending=True)
df_summary

,Unnamed: 0,Model,lr,batch_size,seq_length,week_range,hidden_layer_size,num_layers,lossfunc,mae_validation,mae_validation_1w,mae_validation_2w,mae_validation_3w,mae_validation_4w,Target
0,0,LSTM,0.0001,2,8,64,128,1,SmoothL1Loss(),0.420783,0.311044,0.395818,0.511261,0.591123,rate


##  Forecasting 1000 times

In [ ]:
variable_combinations = []
#summarytable = df_summary
filtered_df_summary = summarytable[summarytable['week_range'] != 32]

# Group by 'region' and then find the row with minimum 'mae_validation' for each group
min_mae_row = filtered_df_summary.loc[filtered_df_summary['mae_validation'].idxmin()]

# Extract the relevant details
variable_combinations= {
    'lr': min_mae_row['lr'],
    'week_range': int(min_mae_row['week_range']),
    'seq_length': int(min_mae_row['seq_length']),
    'hidden_layer_size': int(min_mae_row['hidden_layer_size']),
    'num_layers': int(min_mae_row['num_layers'])
}
variable_combinations

{'lr': 0.0001,
 'week_range': 64,
 'seq_length': 8,
 'hidden_layer_size': 128,
 'num_layers': 1}

In [ ]:

#test_week = '2023-10-01'
#test_week = '2023-11-01'
#test_week = '2023-12-01'
test_week = '2024-01-01'
#test_week = '2024-02-01'
result = LSTM_Prediction(test_week,variable_combinations)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout op

In [ ]:
from datetime import datetime, timedelta


start_date = datetime.strptime('2023-10-07', '%Y-%m-%d')
end_date = datetime.strptime('2024-02-17', '%Y-%m-%d')
current_date = start_date
dates_list = []

while current_date <= end_date:
    dates_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=7)
dates_list
test_weeks = dates_list
for test_week in test_weeks:
    print(test_week)
    file_path = f'/content/drive/MyDrive/Flu Forecasting/retrainLSTM/US result/{test_week}summarytest.csv'

    # Load the summary table, sort it, and filter
    df_summary = pd.read_csv(file_path)
    df_summary = df_summary.sort_values(by='mae_validation', ascending=True)
    filtered_df_summary = df_summary
    min_mae_row = filtered_df_summary.loc[filtered_df_summary['mae_validation'].idxmin()]

    # Extract the relevant details
    variable_combinations= {
        'lr': min_mae_row['lr'],
        'seq_length': int(min_mae_row['seq_length']),
        'hidden_layer_size': int(min_mae_row['hidden_layer_size']),
    }
    result = LSTM_Prediction(test_week,variable_combinations)

Output hidden; open in https://colab.research.google.com to view.